In [ ]:
import geopandas

In [ ]:
fname = 'data/land-polygons-split-4326/land_polygons.shp'
gdf = geopandas.read_file(fname)
len(gdf)

In [ ]:
from rtree import index

# Create an R-tree index
spatial_index = index.Index()

# Insert geometries into the index
for idx, geom in enumerate(gdf.geometry):
    if geom is not None:
        spatial_index.insert(idx, geom.bounds)  # bounds = (minx, miny, maxx, maxy)


In [ ]:
lat0, lon0 = 42.670095, -73.419252
lat1, lon1 = 45.386662, -70.897890
query_bounds = (lon0, lat0, lon1, lat1)
matches = list(spatial_index.intersection(query_bounds))
recs = gdf.iloc[matches]


In [ ]:
import folium

geom = recs.iloc[0].geometry
centroid = geom.centroid
map_center = [centroid.y, centroid.x]
m = folium.Map(location=map_center, zoom_start=10)
folium.GeoJson(geom).add_to(m)
m

In [ ]:
import osmnx

# Define your center and radius (in meters)
lat, lon = 43.063915, -70.705608 
radius = 1000  # meters

# Create a bounding polygon
gdf = osmnx.features_from_point((lat, lon), tags={
    'natural': ['water'],          # lakes, ponds, reservoirs
    'waterway': True,              # rivers, streams, canals
    'landuse': ['reservoir'],      # human-made water areas
    'intermittent': True           # optional: dry season features
}, dist=radius)

# Optional: filter out only polygons (area features)
polygons = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]

# # Save or display
# polygons.plot()


In [ ]:
for rec in gdf.to_dict('records'):
    print(
        rec['name'], rec['waterway'], rec['water'], rec['natural'],
        rec['geometry'].area
    )

In [ ]:
map_center = [lat, lon]
m = folium.Map(location=map_center, zoom_start=10)
for _, row in gdf.iterrows():
    if row['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
        geo_json = folium.GeoJson(row.geometry)
        geo_json.add_to(m)
m


In [ ]:
gdf['gtype'] = [
    z['geometry'].geom_type
    for z in gdf.to_dict('records')
]
gdf[['waterway', 'gtype']].value_counts()

In [ ]:
for p in polygons.to_dict('records'):
    print()
    for key in p:
        if key == 'geometry':
            continue
        if str(p[key]) != 'nan':
            print('%s: %s' % (key, p[key]))


In [ ]:
polygons[['natural', 'water']].value_counts()